<a href="https://colab.research.google.com/github/harjeet88/google_collab/blob/main/spark_exmples/day6_leetcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!chmod -R +x /content/drive/MyDrive/installs/

In [3]:
import os
java_path = '/content/drive/MyDrive/installs/jdk-11.0.2/'
os.environ['JAVA_HOME'] = java_path
os.environ['PATH'] = os.environ['JAVA_HOME'] + '/bin:' + os.environ['PATH']
# Verify installation
!java -version
!which java

openjdk version "11.0.2" 2019-01-15
OpenJDK Runtime Environment 18.9 (build 11.0.2+9)
OpenJDK 64-Bit Server VM 18.9 (build 11.0.2+9, mixed mode)
/content/drive/MyDrive/installs/jdk-11.0.2//bin/java


In [4]:
import pyspark
import os
java_path = '/content/drive/MyDrive/installs/jdk-11.0.2/'
os.environ['JAVA_HOME'] = java_path
os.environ["SPARK_HOME"] = "drive/MyDrive/installs/spark-3.5.1-bin-hadoop3"
os.environ["PYSPARK_PYTHON"] = "python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python3"

In [5]:
import sys
sys.path.append('/content/drive/MyDrive/installs/py_package') # Replace with your actual path

In [6]:
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession
MAX_MEMORY="8g"
maven_coords = [
    "org.apache.spark:spark-avro_2.12:3.2.1",
    "io.delta:delta-core_2.12:2.0.0rc1",
    "org.xerial:sqlite-jdbc:3.36.0.3",
    "graphframes:graphframes:0.8.2-spark3.2-s_2.12",
    "com.acervera.osm4scala:osm4scala-spark3-shaded_2.12:1.0.8",
]

spark = (pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.jars.packages", ",".join(maven_coords))
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.executor.memory", MAX_MEMORY)
    .config("spark.driver.memory", MAX_MEMORY)
    .enableHiveSupport()
    .getOrCreate()
    )

In [8]:
spark

# Find Followers
https://leetcode.com/problems/find-followers-count/?envType=study-plan-v2&envId=top-sql-50

In [9]:
%%writefile /content/drive/MyDrive/data/leetcode_sql180/find_followers.csv
user_id,follower_id
0,1
1,0
2,0
2,1

Writing /content/drive/MyDrive/data/leetcode_sql180/find_followers.csv


In [10]:
from pyspark.sql.functions import *
from os.path import join
from ast import alias

In [12]:
df = spark.read.csv("/content/drive/MyDrive/data/leetcode_sql180/find_followers.csv",inferSchema=True,header=True)

In [13]:
df.show()

+-------+-----------+
|user_id|follower_id|
+-------+-----------+
|      0|          1|
|      1|          0|
|      2|          0|
|      2|          1|
+-------+-----------+



In [15]:
df.groupBy("user_id").agg(count(df.follower_id).alias("followers_count")).orderBy("user_id").show()

+-------+---------------+
|user_id|followers_count|
+-------+---------------+
|      0|              1|
|      1|              1|
|      2|              2|
+-------+---------------+



# Count salary cataegories
https://leetcode.com/problems/count-salary-categories/description/?envType=study-plan-v2&envId=top-sql-50

In [17]:
%%writefile /content/drive/MyDrive/data/leetcode_sql180/count_salary_categories_1907.csv
account_id,income
3,108939
2,12747
8,87709
6,91796

Overwriting /content/drive/MyDrive/data/leetcode_sql180/count_salary_categories_1907.csv


In [18]:
df = spark.read.csv("/content/drive/MyDrive/data/leetcode_sql180/count_salary_categories_1907.csv",inferSchema=True,header=True)

In [19]:
df.show()

+----------+------+
|account_id|income|
+----------+------+
|         3|108939|
|         2| 12747|
|         8| 87709|
|         6| 91796|
+----------+------+



In [23]:
df1 = df.withColumn("category",
    when(col("income")<20000,"Low Salary")
    .when((col("income")>=20000) & (col("income")<=50000),"Average Salary")
    .otherwise("High Salary"))
df1.show()

+----------+------+-----------+
|account_id|income|   category|
+----------+------+-----------+
|         3|108939|High Salary|
|         2| 12747| Low Salary|
|         8| 87709|High Salary|
|         6| 91796|High Salary|
+----------+------+-----------+



In [32]:
df2 = df1.groupBy(col("category")).agg(count("*").alias("accounts_count"))

In [33]:
all_cat= spark.createDataFrame(
    [
        ("Low Salary",0),
        ("Average Salary",0),
        ("High Salary",0)
    ],
    ["category","accounts_count"]
)

In [34]:
all_cat.show()

+--------------+--------------+
|      category|accounts_count|
+--------------+--------------+
|    Low Salary|             0|
|Average Salary|             0|
|   High Salary|             0|
+--------------+--------------+



In [37]:
df2.union(all_cat).groupBy("category").agg(sum("accounts_count").alias("accounts_count")).show()

+--------------+--------------+
|      category|accounts_count|
+--------------+--------------+
|   High Salary|             3|
|    Low Salary|             1|
|Average Salary|             0|
+--------------+--------------+



# Restaurent Growth
https://leetcode.com/problems/restaurant-growth/description/?envType=study-plan-v2&envId=top-sql-50

In [38]:
%%writefile /content/drive/MyDrive/data/leetcode_sql180/restaurent_growth_1321.csv
customer_id,name,visited_on,amount
1,Jhon,2019-01-01,100
2,Daniel,2019-01-02,110
3,Jade,2019-01-03,120
4,Khaled,2019-01-04,130
5,Winston,2019-01-05,110
6,Elvis,2019-01-06,140
7,Anna,2019-01-07,150
8,Maria,2019-01-08,80
9,Jaze,2019-01-09,110
1,Jhon,2019-01-10,130
3,Jade,2019-01-10,150

Writing /content/drive/MyDrive/data/leetcode_sql180/restaurent_growth_1321.csv


In [53]:
df=spark.read.csv("/content/drive/MyDrive/data/leetcode_sql180/restaurent_growth_1321.csv",inferSchema=True,header=True)

In [43]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [41]:
df = df.withColumn("visited_on",to_date(col("visited_on")))
df.show()

+-----------+-------+----------+------+
|customer_id|   name|visited_on|amount|
+-----------+-------+----------+------+
|          1|   Jhon|2019-01-01|   100|
|          2| Daniel|2019-01-02|   110|
|          3|   Jade|2019-01-03|   120|
|          4| Khaled|2019-01-04|   130|
|          5|Winston|2019-01-05|   110|
|          6|  Elvis|2019-01-06|   140|
|          7|   Anna|2019-01-07|   150|
|          8|  Maria|2019-01-08|    80|
|          9|   Jaze|2019-01-09|   110|
|          1|   Jhon|2019-01-10|   130|
|          3|   Jade|2019-01-10|   150|
+-----------+-------+----------+------+



In [66]:
# Calculate daily totals
daily_totals = df.groupBy("visited_on").agg(sum("amount").alias("daily_amount"))\
.orderBy("visited_on")


In [67]:
winSpec = Window.orderBy("visited_on").rowsBetween(-6,0)

In [69]:
daily_totals.withColumn("avg_amount",round(F.avg("daily_amount").over(winSpec),2)).show()

+----------+------------+----------+
|visited_on|daily_amount|avg_amount|
+----------+------------+----------+
|2019-01-01|         100|     100.0|
|2019-01-02|         110|     105.0|
|2019-01-03|         120|     110.0|
|2019-01-04|         130|     115.0|
|2019-01-05|         110|     114.0|
|2019-01-06|         140|    118.33|
|2019-01-07|         150|    122.86|
|2019-01-08|          80|     120.0|
|2019-01-09|         110|     120.0|
|2019-01-10|         280|    142.86|
+----------+------------+----------+



In [70]:
# prompt: offset results from 7th record

# Assuming you want to offset results from the 7th record of the dataframe 'daily_totals'
# in the provided code.

# ... (your existing code) ...

# Calculate daily totals
daily_totals = df.groupBy("visited_on").agg(sum("amount").alias("daily_amount"))\
.orderBy("visited_on")

winSpec = Window.orderBy("visited_on").rowsBetween(-6,0)

# Offset results from the 7th record
offset_daily_totals = daily_totals.withColumn("avg_amount",round(F.avg("daily_amount").over(winSpec),2)).limit(daily_totals.count() - 6)

offset_daily_totals.show()

+----------+------------+----------+
|visited_on|daily_amount|avg_amount|
+----------+------------+----------+
|2019-01-01|         100|     100.0|
|2019-01-02|         110|     105.0|
|2019-01-03|         120|     110.0|
|2019-01-04|         130|     115.0|
+----------+------------+----------+

